# Explore here

It's recommended to use this notebook for exploration purposes.

In [66]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns



## Generar tabla y objetivo

In [70]:
URL = "https://es.wikipedia.org/wiki/Anexo:Películas_con_las_mayores_recaudaciones"
HEADERS = {"User-Agent": "Mozilla/5.0"}  # Evitar bloqueos impersonando el navegador Mozilla.
DB_PATH = "box_office.db"
TABLE_NAME = "peliculas_mas_taquilleras"

def to_number(x):
    """Convierte texto con símbolos de moneda a número entero."""
    if pd.isna(x):
        return pd.NA
    s = str(x)
    s = re.sub(r"[^\d]", "", s)  # deja solo dígitos
    return pd.to_numeric(s, errors="coerce")

def main():
    resp = requests.get(URL, headers=HEADERS, timeout=30)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    target = None
    for t in soup.select("table.wikitable"):
        cap = t.find("caption")
        if cap and "mayores recaudaciones a nivel mundial" in cap.get_text(strip=True).lower() and "usd" in cap.get_text(strip=True).lower():
            target = t
            break
    if target is None:
        raise RuntimeError("No encontré la tabla.")
    df = pd.read_html(str(target), flavor="bs4")[0]

    # limpieza y detección de tipos de columnas.
    for col in df.columns:
        if "recaud" in str(col).lower() or "taquilla" in str(col).lower():
            df[col] = df[col].apply(to_number)

    for name in df.columns:
        ln = str(name).lower()
        if ln in ("n.º", "nº", "n°", "puesto"):
            df[name] = pd.to_numeric(df[name], errors="coerce", downcast="integer")
        if "año" in ln and "estreno" in ln:
            df[name] = pd.to_numeric(df[name], errors="coerce", downcast="integer")
    with sqlite3.connect(DB_PATH) as conn:
        df.to_sql(TABLE_NAME, conn, if_exists="replace", index=False)

print(f"Datos guardados en la tabla '{TABLE_NAME}' de '{DB_PATH}'.")
  


Datos guardados en la tabla 'peliculas_mas_taquilleras' de 'box_office.db'.


## Visualizaciones

In [71]:
import matplotlib.pyplot as plt

plt.style.use("ggplot")

# Detectar columnas
col_world = next((c for c in df.columns if "mundial" in c.lower()), None)
col_movie = next((c for c in df.columns if "película" in c.lower()), None)

if col_world and col_movie:
    top10 = df.nlargest(10, col_world)[[col_movie, col_world]].set_index(col_movie)
    top10[col_world] = top10[col_world] / 1e9  # convertir a miles de millones USD
    top10.plot(kind="barh", figsize=(10, 6), color="steelblue")
    plt.title("Top 10 películas más taquilleras (en miles de millones USD)")
    plt.xlabel("Recaudación mundial (B USD)")
    plt.ylabel("")
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    print("No se encontraron columnas de 'mundial' o 'película'.")

# Recaudación mundial media por año
col_year = next((c for c in df.columns if "año" in c.lower()), None)
if col_year and col_world:
    yearly = df.groupby(col_year)[col_world].mean().dropna() / 1e9
    yearly.plot(kind="line", marker="o", figsize=(10, 5))
    plt.title("Evolución de la recaudación media por año de estreno")
    plt.ylabel("Media recaudación (B USD)")
    plt.xlabel("Año de estreno")
    plt.tight_layout()
    plt.show()
else:
    print("No se encontró columna de 'año' o 'recaudación mundial'.")

# Comparación taquilla doméstica vs internacional
domestic_col = next((c for c in df.columns if "ee. uu" in c.lower()), None)
foreign_col = next((c for c in df.columns if "fuera" in c.lower()), None)

if domestic_col and foreign_col and col_movie:
    comp = df.nlargest(10, col_world)[[col_movie, domestic_col, foreign_col]]
    comp = comp.set_index(col_movie) / 1e9
    comp.plot(kind="barh", stacked=True, figsize=(10, 6))
    plt.title("Top 10: taquilla EE. UU. vs fuera de EE. UU. (B USD)")
    plt.xlabel("Recaudación total (B USD)")
    plt.ylabel("")
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    print("No se encontraron columnas para taquilla doméstica/internacional.")

NameError: name 'df' is not defined